In [97]:
#will need to parse an arbitrary svg, find ptextpaths, and prettify them.
from vec2d import Vec2D
import xml.etree.ElementTree as etree
import math
from cprettypath import PrettyPath
svgfile='textpath.svg'
svg=etree.parse(svgfile)
ns={'s':'http://www.w3.org/2000/svg','x':'http://www.w3.org/1999/xlink'}



def getlabel(tp):
    return tp.text#duh
def getpath(tp):
    href='{http://www.w3.org/1999/xlink}href'
    pathid=tp.get(href)
    return svg.getroot().find(".//s:path[@id='"+pathid[1:]+"']",ns)
def tapered_from_path(pth,text):
    d=pth.get('d')
    #don't care about curves, we'll replace them anyway. just care about start_width,
    #start_theta, end_width, and end_theta. all from linear segments. so we do care about curves a bit
    #inkscape is good about commas
    d=d.split(' ')
    absolute=False
    if d.pop(0)=='M':
        absolute=True
    #gonna make some assumptions here, hope they don't bite me. watch out
    def v_fr_s(s):
        return Vec2D(*tuple(float(a) for a in s.split(',')))
    p0=v_fr_s(d.pop(0))
    here=p0
    try:
        s=d.pop(0)
        p1=v_fr_s(s)
    except ValueError as e:#means there was an l here. just move on.
        if s=='l':
            absolute=False
        elif s=='L':
            absolute=True
        else:
            raise e
        p1=v_fr_s(d.pop(0))
    if not absolute:
        p1=here+p1
    here=p1
    #next bit has to be a curve cuz i say so
    s=d.pop(0)
    if s=='c':
        absolute=False
    elif s=='C':
        absolute=True
    else:
        raise ValueError
    #we ignore control points
    d.pop(0)#p1
    d.pop(0)#p2
    p2=v_fr_s(d.pop(0))
    if not absolute:
        p2=here+p2
    here=p2
    #next will be an l
    s=d.pop(0)
    if s=='l':
        absolute=False
    elif s=='L':
        absolute=True
    else:
        raise ValueError
    p3=v_fr_s(d.pop(0))
    if not absolute:
        p3=here+p3
    
    start=(p0+p1)*0.5
    end=(p2+p3)*0.5
    sn=p1-p0
    start_width=sn.r
    en=p3-p2
    end_width=en.r
    if (sn).cross(p2-p1) > 0:#0-1-2-3 counter-clockwise
        start_theta=sn.t+math.pi/2
        end_theta=en.t-math.pi/2
    else:
        start_theta=sn.t-math.pi/2
        end_theta=en.t+math.pi/2
    path=PrettyPath(start,Vec2D(r=1,t=start_theta),0,0)
    path.optimize(end=end,t0=start_theta,t1=end_theta,segs=5)
    taper=path.taperedpath(5,start_width,end_width=end_width,fill="#008000")
    svgtext=''
    if text:
        svgtext=None
        while svgtext is None:
            txpth=TextPath(text,start_width,end_width,path,5)
            svgtext=txpth.text_to_svg()
            if svgtext==None:#too wide for path.
                start_width*=0.9
                end_width*=0.9
    else:
        text=''
    return '<g id="'+pth.get('id')+text+'" >'+taper.tostring()+svgtext+'</g>'
    
#do the business:
textpaths=svg.getroot().findall('.//s:textPath',ns)
for tp in textpaths:
    text=tp[0].text#in a tspan
    pth=getpath(tp)
    replacement=tapered_from_path(pth,text)
#now delete the textpath and its parent text element replace the path with our replacement
    tpid=tp.get('id')
    parent=svg.getroot().find(".//s:textPath[@id='"+tpid+"']/../..",ns)
    deletable=parent.find(".//s:textPath/[@id='"+tpid+"']/..",ns)
    parent.remove(deletable)
    pathparent=svg.getroot().find(".//s:path[@id='"+pth.get('id')+"']/..",ns)
    pathparent.remove(pth)
    pathparent.insert(0,etree.fromstring(replacement))


In [96]:
with open('arcpath.svg','w') as f:
    f.write(etree.tostring(ap.getroot(),encoding='unicode'))

In [95]:
ap=etree.parse('arcpath.svg')
etree.tostring(ap.getroot(),encoding='unicode')

'<ns0:svg xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.w3.org/2000/svg" xmlns:ns1="http://www.inkscape.org/namespaces/inkscape" xmlns:ns2="http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd" xmlns:ns4="http://creativecommons.org/ns#" xmlns:ns6="http://www.w3.org/1999/xlink" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" height="297mm" id="svg256186" version="1.1" viewBox="0 0 744.09448819 1052.3622047" width="210mm" ns2:docname="arcpath.svg" ns1:version="0.91 r13725">\n  <ns0:defs id="defs256188" />\n  <ns2:namedview bordercolor="#666666" borderopacity="1.0" id="base" pagecolor="#ffffff" showgrid="false" ns1:current-layer="layer1" ns1:cx="244.09247" ns1:cy="520" ns1:document-units="px" ns1:pageopacity="0.0" ns1:pageshadow="2" ns1:window-height="1519" ns1:window-maximized="1" ns1:window-width="2880" ns1:window-x="0" ns1:window-y="0" ns1:zoom="1.8142578" />\n  <ns0:metadata id="metadata256191">\n    <rdf:RDF>\n      <ns4:Work rdf:about="">\n        <dc:forma

In [73]:
from vec2d import *
from cprettypath import radians
import math as np
import subprocess
import xml.etree.ElementTree as etree
from cprettypath import PrettyPath,get_point
class TextPath():
    def __init__(self,text,start_width,end_width,path,segs,fill='#f87089'):
        self.text=text
        self.sw=start_width
        self.ew=end_width
        self.path=path
        self.segs=segs
        self.fill=fill
    def text_to_svg(self):
        with open('/tmp/text.svg','w') as f:
            f.write('<svg><text font-size="20px" font-family="Vim SM">'+self.text+'</text></svg>')  
        r=subprocess.run(['/usr/bin/inkscape','-ST','/tmp/text.svg','-l','/tmp/text.svg'],stdout=subprocess.PIPE,universal_newlines=True)

        height=float(r.stdout.split('\n')[1].split(',')[4])
        width=float(r.stdout.split('\n')[1].split(',')[3])
        #self.path.t0.r=1+width*self.sw*(np.exp((self.ew-self.sw)/height*(1))-1)/(self.ew-self.sw)


        textpaths=etree.parse('/tmp/text.svg').getroot()[2]
        paths=[]
        for path in textpaths:
            paths.append([[float(j) for j in i.split(',')] if (',' in i) else i for i in path.get('d').split(' ')])
        #separates paths into a command list
        
        #print(paths)    
        for k in range(len(paths)):
            here=[0,0]
            prev=[0,0]
            subpath_start=[0,0]
            curve=False
            moveto=False
            clozed=False
            curvei=0
            for i in range(len(paths[k])):
                if isinstance(paths[k][i],list):#means a relative (x,y)
                    if absolute:
                        here[0]=paths[k][i][0]
                        here[1]=paths[k][i][1]
                    else:
                        here[0]=prev[0]+paths[k][i][0]
                        here[1]=prev[1]+paths[k][i][1]
                    
                    tfm=self.do_transform(here,height,width)
                    if tfm is None:#path too short
                        return None
                    else:
                        paths[k][i]=tfm
                    
                    paths[k][i]=','.join(str(j) for j in paths[k][i])
                    if curve:#we're parsing a bezier point
                        curvei=curvei+1
                        if curvei==3:#the destination of the curve
                            prev[0]=here[0]
                            prev[1]=here[1]
                            curvei=0
                    else:
                        prev[0]=here[0]
                        prev[1]=here[1]
                    if moveto:
                        subpath_start[0]=here[0]
                        subpath_start[1]=here[1]
                        moveto=False
                else:
                    if paths[k][i]==paths[k][i].upper():
                        absolute=True
                    else:
                        absolute=False
                        paths[k][i]=paths[k][i].upper()
                    curve=False
                    moveto=False
                    if paths[k][i]=='C':
                        curve=True
                        curvei=0
                        clozed=False
                    elif paths[k][i]=='Z':
                        clozed=True
                        #prev[0]=subpath_start[0]
                        #prev[1]=subpath_start[1]
                        #here[0]=subpath_start[0]
                        #here[1]=subpath_start[1]
                    elif paths[k][i]=='M':
                        moveto=True
                        if clozed:
                            pass
                            #prev[0]=0
                            #prev[1]=0
                            #here[0]=0
                            #here[1]=0
                        clozed=False
                    else:
                        clozed=False
            paths[k]=' '.join(paths[k])

        ret='<g id="text">'
        for path in paths:
            ret+='<path d="'+path+'" style="fill:'+self.fill+'"/>'
        ret+='</g>'
        return(ret)
    def do_transform(self,xy,height,width):
        x,y=xy
        #x+=100
        y+=height/2
        x_p=self.sw*(np.exp((self.ew-self.sw)/height*(x/width))-1)/(self.ew-self.sw)
        y_p=y*(self.sw+(self.ew-self.sw)*x_p)/height
        x_p*=width
        if x_p>1:#path not long enough
            return None
        x_p,y_p=self.on_path(x_p,y_p)
        
        return x_p,y_p
    def on_path(self,x,y):
        #print('x:',x)
        pathpoint=get_point(self.path,self.segs,x)
        tan=Vec2D(r=y,t=pathpoint[1]+radians(90))
        #tan._update_car()
        xy=pathpoint[0]+tan
        #return x,y
        return xy.x,xy.y